In [ ]:
# This notebook contains the program for reading the relevant information from all XML files

In [ ]:
import os
import numpy as np
import xml.etree.cElementTree as et
import pandas as pd
from IPython.display import display, HTML
path = r"N:\AUG-ImageRepository\loc_img_db\192\2013-01-14"

In [ ]:
# series_complete = complete_series(root, tree)
# study_table = get_study_table(root, tree)
path1 = r"N:\AUG-ImageRepository\loc_img_db\192\2013-01-14"
path = r"N:\AUG-ImageRepository\loc_img_db\8244\2002-01-11"
file_path = os.path.join(path, 'export.xml')
tree = et.ElementTree()
tree.parse(file_path)
root = tree.getroot()
patient = get_patient(root)
get_study_table(root, tree)
#get_explorative_data(root, tree)
#get_explorative_data(root, tree)

In [ ]:
# below methods will later be defines in a class object for formlaised Heidelberg XML parsing program. 

In [ ]:
def get_parentmap(tree):
    return(dict((c, p) for p in tree.getiterator() for c in p))

def get_patient(root):
    BODY = root[0]
    for child in BODY:
        if 'Patient' in str(child):
            patient = child
    return(child)

def get_study(root):
    patient = get_patient(root)
    for child in patient:
        if 'Study' in str(child):
            study = child
    return(study)

def get_series(root):
    series = []
    study = get_study(root)
    for child in study:
        if 'Series' in str(child):
            series.append(child)
    return(series)

def get_patient_id(patient):
    for child in patient:
        if 'PatientID' in str(child):
            patientID = child
    return patientID

def pID_orNAN(patient):
    'This function returns the PatientID if exists otherwise Nan'
    l = []
    r = []
    for child in patient:
        if "PatientID" in str(child):
            l.append(child.text)
        if "PatientID" not in str(child):
            l.append("No PatientID")
        for i in l:
            if "No" not in i:
                r.append(i)      
    if not r:
        r.append("NaN")
    return(r[0])

def get_study_id(study):
    for child in study:
        if 'ID' in str(child) and 'Study' not in str(child):
            studyID = child
    return studyID

def get_series_id(series):
    for child in series:
        if 'ID' in str(child) and 'Series' not in str(child) and 'Prog' not in str(child):
            seriesID = child
    return seriesID

def get_patient_birthdate(root):
    patient = get_patient(root)
    for child in patient:
        if 'Birthdate' in str(child):
            date = child[0]
            for children in date:
                if 'Year' in str(children):
                    year = children.text
                if 'Month' in str(children):
                    month = children.text
                if 'Day' in str(children):
                    day = children.text
    
    return('-'.join((year, month, day)))

def get_study_date(root):
    study = get_study(root)
    for child in study:
        if 'StudyDate' in str(child):
            date = child[0]
            for children in date:
                if 'Year' in str(children):
                    year = children.text
                if 'Month' in str(children):
                    month = children.text
                if 'Day' in str(children):
                    day = children.text
    
    return('-'.join((year, month, day)))

def get_patient_table(root):
    patient =  get_patient(root)
    for child in patient:
        if "Sex" in str(child):
            sex = child.text
        if "ID" in str(child) and "Patient" not in str(child):
            ID = child.text
    PatientID = pID_orNAN(patient)

    date = get_patient_birthdate(root)
    d = {'Patient_ID': [ID],'PatientID_DB': [PatientID], 'sex': [sex], 'date':[date]}
    return(pd.DataFrame(data = d))

def get_study_table(root, tree):
    PatientID = []
    study_date = get_study_date(root)
    parent_map = get_parentmap(tree)
    study = get_study(root)
    for child in study:
        if 'ID' in str(child) and 'Study' not in str(child):
            study_ID = child.text
            patient_id = parent_map[parent_map[child]]
            #print(patient_id)
            PatientID.append(patient_id.findall('ID')[0].text)

    study_d = {'Patient_ID': PatientID,'Study_ID': [study_ID], 'study_date':[study_date]}
    return(pd.DataFrame(data = study_d))
def get_thickness_grid(root):
    thickness_grids = []
    series = get_series(root)
    for ser in series:
        for child in ser:
            if 'ThicknessGrid' in str(child):
                thickness_grids.append(child)
    return(thickness_grids)
def get_thicknessgrid(root, tree):

    Type = []
    CentralThickness = []
    MinCentralThickness = []
    MaxCentralThickness = []
    TotalVolume = []
    X_cetner_pos = []
    Y_cetner_pos = []
    Series_ID = []
    parent_map = get_parentmap(tree)
    series = get_series(root)
    for ser in series:
        for child in ser:
            if 'ThicknessGrid' in str(child):
                series_id = parent_map[child]
                Series_ID.append(series_id.findall('ID')[0].text)
                #get direct children
                Type.append(child.findall('Type')[0].text)
                CentralThickness.append(child.findall('CentralThickness')[0].text)
                MinCentralThickness.append(child.findall('MinCentralThickness')[0].text)
                MaxCentralThickness.append(child.findall('MaxCentralThickness')[0].text)
                TotalVolume.append(child.findall('TotalVolume')[0].text)

                #get center position
                center_pos = child.findall('CenterPos')[0]
                for children in center_pos:
                    X_cetner_pos.append(children[0].text)
                    Y_cetner_pos.append(children[1].text)
    
    thicknessgrip_d = {'Series_ID': Series_ID, 'Type':Type,'CentralThickness':CentralThickness,\
                       'MinCentralThickness': MinCentralThickness,\
                    'MaxCentralThickness': MaxCentralThickness,'TotalVolume': TotalVolume,\
                    'X_cetner_pos':X_cetner_pos,'Y_cetner_pos':Y_cetner_pos}
    
    return(pd.DataFrame(data = thicknessgrip_d))

def get_thicknessgrid_zones(root, tree):
    Name = []
    AvgThickness = []
    Volume = []
    Series_ID = []
    thick_grid = get_thickness_grid(root)
    parent_map = get_parentmap(tree)
    for child in thick_grid:
        for children in child:
            if 'Zone' in str(children):
                series = parent_map[parent_map[children]]
                Series_ID.append(series.findall('ID')[0].text)
                Name.append(children.findall('Name')[0].text)
                AvgThickness.append(children.findall('AvgThickness')[0].text)
                Volume.append(children.findall('Volume')[0].text)

    col = ['Series_ID','Name','AvgThickness','Volume']
    ThicknessGrid_df = pd.DataFrame([Series_ID,Name, AvgThickness, Volume]).T
    ThicknessGrid_df.columns = col
    return(ThicknessGrid_df)

def get_series_table(root, tree):
    series_UID = []
    series_ID = []
    Modality = []
    Laterality = []
    series_type = []
    NumImages = []
    ModalityProcedure = []
    Study_ID = []
    parent_map = get_parentmap(tree)
    study = get_study(root)
    for child in study:
        if 'Series' in str(child):
            for children in child:                
                if 'SeriesUID' in str(children):
                    series_UID.append(children.text)
                if 'ID' in str(children) and 'Series' not in str(children) and "Prog" not in str(children):
                    series_ID.append(children.text)
                    #get the study ID
                    study_id = parent_map[child]
                    Study_ID.append(study_id.findall('ID')[0].text)
                if 'Modality' in str(children) and 'Procedure' not in str(children):
                    Modality.append(children.text)
                if 'ModalityProcedure' in str(children):
                    ModalityProcedure.append(children.text)
                if 'Type' in str(children):
                    series_type.append(children.text)
                if 'Laterality' in str(children):
                    Laterality.append(children.text)
                if 'NumImages' in str(children):
                    NumImages.append(children.text)

    series_d = {'Study_ID': Study_ID, 'series_ID': series_ID, 'series_UID':series_UID,'Modality': Modality,\
                'ModalityProcedure':ModalityProcedure, \
                'series_type': series_type,'laterality':Laterality, 'NumImages': NumImages}

    series_pd = pd.DataFrame(data = series_d)
    #thickness_pd = get_thicknessgrid(root, tree)

    return(series_pd)

def complete_series(root, tree):
    t_grid = get_thicknessgrid(root, tree)
    z_grid = get_thicknessgrid_zones(root, tree)
    s_table = get_series_table(root, tree)
    #display(t_grid, z_grid)
    tz_grid = t_grid.merge(z_grid, left_on = 'Series_ID', right_on = 'Series_ID', how = 'left')
    #display(s_table, tz_grid)
    full_series = s_table.merge(tz_grid, left_on = 'series_ID', right_on = 'Series_ID', how = 'left')

    return(full_series)

def get_explorative_data(root, tree):
    series_complete = complete_series(root, tree)
    study_table = get_study_table(root, tree)
    patient_table = get_patient_table(root)
    #display(study_table,series_complete)
    #series_complete = series_complete.T.drop_duplicates().T
    study_series = study_table.merge(series_complete, left_on = 'Study_ID', right_on = 'Study_ID', how = 'left')
    study_series = study_series.T.drop_duplicates().T
    pat_st_ser = patient_table.merge(study_series, left_on = 'Patient_ID', right_on = 'Patient_ID',how = 'left')
    #pat_st_ser = pat_st_ser.T.drop_duplicates().T
    return(pat_st_ser)

def get_patient_full_key_table(root, tree):
    #initialize lists for storage
    patient_col = []
    patient_values = []
    study_col = []
    study_values = []
    series_col = []
    series_values = []
    images_col = []
    images_values = []
    get_patient_table(root)
    parent_map = get_parentmap(tree)
    patient = get_patient(root)
    for child in patient.getiterator():
        if 'ID' in str(child):
            #get patient ID table
            if 'Patient' in parent_map[child].tag:
                patient_col.append(['P_'+child.tag])
                patient_values.append([child.text])

            if 'Study' in parent_map[child].tag:
                #takes ID columns from child nodes to Study and creates a table
                study_col.append(['Study_'+child.tag])
                study_values.append([child.text])
                #get corresponding patient ID and appends to list
                p = parent_map[parent_map[child]]
                PatientID = get_patient_id(p)
                study_col.append([PatientID.tag])
                study_values.append([PatientID.text])

            if 'Series' in parent_map[child].tag and 'Reference' not in parent_map[child].tag \
            and 'Prog' not in str(child):
                series_col.append(['Series_'+child.tag])
                series_values.append([child.text])
                s = parent_map[parent_map[child]]
                s_id = get_study_id(s)
                series_col.append(['Study_'+s_id.tag])
                series_values.append([s_id.text])

        if 'Image' in parent_map[child].tag and ('Laterality' in str(child) or 'ID' in str(child)):
            #print(parent_map[child].tag)
            images_col.append(['Image_'+child.tag])
            images_values.append([child.text])
            #get the series ID for the Im table
            ser = parent_map[parent_map[child]]   
            ser_id = get_series_id(ser)
            images_col.append(['Series_'+ser_id.tag])
            images_values.append([ser_id.text])

    #prepare the patient table
    columns = [item for sublist in patient_col for item in sublist]
    values = [item for sublist in patient_values for item in sublist]
    patient_df = pd.DataFrame(values).T
    patient_df.columns = columns

    #prepare the study table, with patient ID
    columns = [item for sublist in study_col for item in sublist]
    values = [item for sublist in study_values for item in sublist]
    study_df = pd.DataFrame(values).T
    study_df.columns = columns
    study_df = study_df.T.drop_duplicates().T

    #prepare series table, with study ID , TODO: remove creation of duplicate study ID columns
    columns = [item for sublist in series_col for item in sublist]
    values = [item for sublist in series_values for item in sublist]
    #get dimensions to reshape list/array objects
    number_of_rows = columns.count('Series_ID')
    number_of_columns = int(len(columns)/number_of_rows)
    cols = columns[0:number_of_columns]
    vals = np.asarray(values).reshape(number_of_rows,number_of_columns)
    #convert to data frame
    series_df = pd.DataFrame(vals)
    series_df.columns = cols
    series_df = series_df.T.drop_duplicates().T

    #prepare Image table with Series ID
    columns = [item for sublist in images_col for item in sublist]
    values = [item for sublist in images_values for item in sublist]
    number_of_rows = columns.count('Image_ID')
    number_of_columns = int(len(columns)/number_of_rows)


    cols = columns[0:number_of_columns]
    vals = np.asarray(values).reshape(number_of_rows,number_of_columns)
    images_df = pd.DataFrame(vals)
    images_df.columns = cols
    images_df = images_df.T.drop_duplicates().T

    #display(patient_df, study_df, series_df, images_df)

    #merge all tables into one full key table
    patient_study = patient_df.merge(study_df, left_on = 'P_PatientID', \
                                     right_on = 'PatientID', how='outer' )
    patient_study_series = patient_study.merge(series_df, left_on = 'Study_ID', \
                                               right_on = 'Study_ID', how = 'outer')
    patient_key_table = patient_study_series.merge(images_df, left_on = 'Series_ID', \
                                                   right_on = 'Series_ID', how = 'outer')
    #display(patient_key_table)
    return(patient_key_table)


In [ ]:
import os
write_path = r'Holmberg, Olle\explorative.csv'
os.getcwd()
path = r'N:\AUG-ImageRepository\loc_img_db'
folder = os.listdir(path)
df_list = []
k = 0
i = 0 
for sub_folder in folder:
    #print(sub_folder)
    
    sub_path = os.path.join(path, sub_folder)
    sub_dir = os.listdir(sub_path)
    #print(sub_dir)
    for sub_files in sub_dir:
        #print(sub_files)
        scnd_sub_dir = os.path.join(sub_path, sub_files)
        #print(scnd_sub_dir)
        scnd_sub_files = os.listdir(scnd_sub_dir)
        #print(scnd_sub_files)
        for third_sub_files in scnd_sub_files:
            #print(third_sub_files)
            if third_sub_files.endswith('.xml') and 'corrected' not in str(third_sub_files):
                #print("one xml file in {}!".format(file_path))
                try:
                    print(i)
                    i +=1
                    xml_path = os.path.join(scnd_sub_dir,third_sub_files)
                    tree = et.ElementTree()
                    tree.parse(xml_path)
                    root = tree.getroot()
                    df = get_explorative_data(root, tree)
                    #display(df)
                    df_list.append(df)
                except:
                    k += 1
                    print("Number of corrupt files are{}".format(k))
                    print("Last processed file was {}".format(third_sub_files))


#get column names to reset order after concat
col = ['PatientID_DB','Patient_ID', 'date', 'sex', 'Study_ID', 'study_date', 'Modality'\
       ,'ModalityProcedure', 'NumImages', 'laterality', 'series_ID', 'series_UID'\
       ,'series_type', 'CentralThickness', 'MaxCentralThickness', 'MinCentralThickness'\
       ,'Series_ID', 'TotalVolume', 'Type', 'X_cetner_pos', 'Y_cetner_pos', 'Name'\
       ,'AvgThickness', 'Volume']
df = pd.concat(df_list)[col]
display(df)
df.to_csv('explorative.csv')